In [1]:
import gmaps
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

# Google developer API key
from api_keys import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [2]:
#introducing Vehicle data
vehicle_df = pd.read_csv("Data/vehicle_data.csv.gz", compression="gzip")

#take care of NaN velue
vehicle_df = vehicle_df.dropna()

#creating dataframe for car's for selective years
vehicle_year_df = vehicle_df.loc[vehicle_df['model_year'].astype(int) <= 2020]
#vehicle_year_df.head()

In [3]:
#calculating vehicle age till 2009 by model year 
veh_age = 2020 - vehicle_year_df['model_year'].astype(int)

#creating new data frame with new column for vehicle_age
new_vehicle_df = pd.DataFrame(vehicle_year_df)
new_vehicle_df['vehicle_age'] = veh_age
#new_vehicle_df.head()

In [4]:
#introducing zip code coordinates
zip_df = pd.read_csv('Data/us-zip-code-latitude-and-longitude.csv')
zip_df = zip_df.rename (columns = {"Zip" : "zip"})

#take care of NaN velue
zip_df = zip_df.dropna()
#zip_df.head()

In [5]:
# find mean of vehicle age by zip code
veh_age_mean_df = new_vehicle_df.groupby('zip').agg({'vehicle_age':'mean'})
veh_age_mean_df = veh_age_mean_df.reset_index()
veh_age_mean_df.head()

,zip,vehicle_age
0,0,11.0
1,646,37.0
2,680,4.0
3,727,14.0
4,739,7.0


In [6]:
#add zip code coordinates to vehicle age mean dataframe
vehicle_age_2020_df = pd.merge(veh_age_mean_df, zip_df, on=['zip','zip'], how="left")

#take care of NaN velue
vehicle_age_2020_df = vehicle_age_2020_df.dropna()
vehicle_age_2020_df.head()

,zip,vehicle_age,City,State,Latitude,Longitude,Timezone,Daylight savings time flag
1050,10001,6.330077,New York,NY,40.750742,-73.99653,-5.0,1.0
1051,10002,8.041483,New York,NY,40.717040,-73.98700,-5.0,1.0
1052,10003,7.948633,New York,NY,40.732509,-73.98935,-5.0,1.0
1054,10005,5.339164,New York,NY,40.706019,-74.00858,-5.0,1.0
1055,10006,5.532009,New York,NY,40.707904,-74.01342,-5.0,1.0


In [7]:
#Plot Heatmap for vehicle age by zip code
locations = vehicle_age_2020_df[["Latitude", "Longitude"]].astype(float)
vehicle_age = vehicle_age_2020_df["vehicle_age"].astype(float)


#create heat layer for vehicle age by zip
heat_layer = gmaps.heatmap_layer(locations, weights=vehicle_age, 
                                  dissipating=False, max_intensity=500,                                  
                                  point_radius=0.20)


#combine both maps
fig = gmaps.figure(map_type="HYBRID")
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))